# Paris 2024 Olympic Medal Contender Analysis

### Approach
I will use data from worldatheltics.org to predict the most likely medalists for the Paris 2024 Olympics for select track and field events. I will use the data from the past two years for each qualifying athlete to predict their performance in the 2024 Olympics. The predictive model will consist of two parts: (1) a non-linear function (likely a Gompertz growth model) to model the athlete's performance trajectory (e.g., improving, declining, stable) going into the Paris Olympics and (2) a Monte Carlo simulation to account for uncertainty in the predictions. The non-linear model will limit the athlete's performance as they approach/surpass their peak performance (personal best or PB) -- the closer to their PB they are, . The Monte Carlo simulation will use the athlete's consistency from event to event to determine the uncertainty in the athlete's performance in 2024.


- For each event, extract the performance data for each event by qualifying athlete for 2023 and 2024. This will allow us to analyse and compare the athletes' performances in the two years leading up to the Olympics.
- Correct for wind conditions in the performance data for events where wind conditions can affect performance (e.g. track events).
- Model athlete performance data to predict the top athletes for each event in 2024.
- Modelling will involve determining an athlete's trajectory (e.g. improving, declining) relative to their personal best performance. The trajectory will be converted to a non-linear function (Gompertz growth model) to limit the athlete's performance as they approach/surpass their peak performance.
- Modelling will also involve determining an athlete's consistency from event to event. The consistency will be used to determine the uncertainty in the athlete's performance in 2024 and incorporated as a parameter in the Monte Carlo simulation (see below).
- The trajectory and consistency will be used to predict the athlete's performance in 2024 using a Monte Carlo simulation to account for uncertainty in the predictions.

In [144]:
import os
import pandas as pd
import requests # currently not using
from bs4 import BeautifulSoup # currently not using

from selenium import webdriver
import chromedriver_autoinstaller
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException


We'll start by extracting the eventIDs and event names which are used throughout the website and the analysis. This will allow us, for example to loop through extractions of data from the different events which have separate websites, but the same web and data structure.

In [92]:
# The page contains a a lot of JavaScript code that generates the content dynamically
# We can use Selenium to simulate a browser and get the content
chromedriver_autoinstaller.install()

chrome_options = Options()
chrome_options.add_argument("--headless")  # Use --headless for older versions
browser = webdriver.Chrome(options=chrome_options)

url = "https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229509" # this is the URL for the womens 100m event qualifiers for the 2024 Paris Olympics
browser.get(url)

# Wait for the page and all of its elements to load
browser.implicitly_wait(30)

# Isolate the the eventID data (value and name) from the dropdown menu with the tag name "select"
eventID_data = browser.find_element(By.TAG_NAME, "select")

eventID_df = pd.DataFrame(columns=["eventID", "Event_Name"]) # Create an empty dataframe to store the event_ID codes

# Loop through the options in the dropdown menu and store the eventID and event name in the dataframe 
for option in eventID_data.find_elements(By.TAG_NAME, "option"):
    eventID = option.get_attribute("value")
    event_name = option.text
    eventID_df = pd.concat([eventID_df, pd.DataFrame([{"eventID" : eventID, "Event_Name" : event_name}])], ignore_index=True)   # note that we use pd.concat to append the 
                                                                                                                                # new data to the dataframe since the append 
                                                                                                                                # method has been deprecated from newer 
                                                                                                                                # versions of pandas

# Remove the first row of the dataframe as it contains the default value of the dropdown menu
eventID_df = eventID_df.iloc[1:]

# Close the browser
browser.quit()

# Print the dataframe
eventID_df



,eventID,Event_Name
1,10229509,Women's 100 Metres
2,10229510,Women's 200 Metres
3,10229511,Women's 400 Metres
4,10229512,Women's 800 Metres
5,10229513,Women's 1500 Metres
6,10229514,Women's 5000 Metres
7,10229521,"Women's 10,000 Metres"
8,10229522,Women's 100 Metres Hurdles
9,10229523,Women's 400 Metres Hurdles
10,10229524,Women's 3000 Metres Steeplechase


### Step 1.B: Adding URLs to the EventID Dataframe to extract qualifying athlete names for each event

Now that we have the eventIDs and event names, we can start extracting the qualifying athletes for each event. First, we'll add an additional column to the eventID_df dataframe to store the URL for each event. This will allow us to loop through the events and extract the data for each event.

In [93]:
# Add an extra column to the eventID_df called "Qual_URLs" to store the URLs of the olympics qualification data
eventID_df["Qual_URLs"] = ""

# Loop through the eventID_df and store the URLs of the olympics qualification data in the "Qual_URLs" column
for index, row in eventID_df.iterrows():
    eventID = row["eventID"]
    url = f"https://worldathletics.org/stats-zone/road-to/7153115?eventId={eventID}"
    eventID_df.at[index, "Qual_URLs"] = url

# Print the dataframe in wide format so that the URLs are visible
pd.set_option("display.max_colwidth", None)
eventID_df


,eventID,Event_Name,Qual_URLs
1,10229509,Women's 100 Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229509
2,10229510,Women's 200 Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229510
3,10229511,Women's 400 Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229511
4,10229512,Women's 800 Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229512
5,10229513,Women's 1500 Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229513
6,10229514,Women's 5000 Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229514
7,10229521,"Women's 10,000 Metres",https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229521
8,10229522,Women's 100 Metres Hurdles,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229522
9,10229523,Women's 400 Metres Hurdles,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229523
10,10229524,Women's 3000 Metres Steeplechase,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229524


NOTE: clean up below

In [117]:
# Quick housekeeping before the next step:
# Remove apostrophes and commas from the "Event_Name" and replace spaces with underscores to create valid variable names
eventID_df["Event_Name"] = eventID_df["Event_Name"].str.replace("'", "").str.replace(" ", "_").str.replace(",", "")
# Error check the changes
eventID_df["Event_Name"]

# We'll also create a column with dataframe names for each event that we'll use later
# The dataframe names will follow the format "Qaul_athletes_{Event_Name}_df"
eventID_df["df_name"] = "Qual_athletes_" + eventID_df["Event_Name"] + "_df"

# Print the dataframe to check the changes
eventID_df

# We're going to save the qualifier data for each event to a CSV file in the CSV_dataframes folder in the working directory
# So, we'll create the folder if it does not exist (e.g. if you're running the notebook separately)
if not os.path.exists("CSV_dataframes"): # Create the CSV_dataframes folder if it does not exist
    os.makedirs("CSV_dataframes")


,eventID,Event_Name,Qual_URLs,df_name
1,10229509,Womens_100_Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229509,Qual_athletes_Womens_100_Metres_df
2,10229510,Womens_200_Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229510,Qual_athletes_Womens_200_Metres_df
3,10229511,Womens_400_Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229511,Qual_athletes_Womens_400_Metres_df
4,10229512,Womens_800_Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229512,Qual_athletes_Womens_800_Metres_df
5,10229513,Womens_1500_Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229513,Qual_athletes_Womens_1500_Metres_df
6,10229514,Womens_5000_Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229514,Qual_athletes_Womens_5000_Metres_df
7,10229521,Womens_10000_Metres,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229521,Qual_athletes_Womens_10000_Metres_df
8,10229522,Womens_100_Metres_Hurdles,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229522,Qual_athletes_Womens_100_Metres_Hurdles_df
9,10229523,Womens_400_Metres_Hurdles,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229523,Qual_athletes_Womens_400_Metres_Hurdles_df
10,10229524,Womens_3000_Metres_Steeplechase,https://worldathletics.org/stats-zone/road-to/7153115?eventId=10229524,Qual_athletes_Womens_3000_Metres_Steeplechase_df


### Step 2: Extracting the Qualifying Athletes for each event

We'll iterate through the eventIDs df and extract the qualifying athletes for each event from the table on the website. We'll store the data separate dataframes for each event with the following naming convention: athletes_{event_name}_df. Each row will represent an athlete and will contain the following columns: "QP" (Qualifying Position), "Nat" (country), "Athlete", "Status" (how they qualified), "Details" (details about their qualifying performance).

The qualification table on the website also includes athletes that qualified but are not part of the final team, because of per country limits. These athletes do not have a number in the "QP" column. We can use an argument later to parse out the athletes that are part of the final team.

**Approach:**
We'll use Selenium again to scrape the table data from the website, since the data is not available as a direct html download due to JavaScript used to load the table data on the website.

#### Issues to resolve

The 'StaleElementReferenceException' error comes up intermittently. This occurs when the page is reloaded and the target element ("tbody", "tr", or "td") is no longer attached to the DOM (see: https://stackoverflow.com/questions/27003423/staleelementreferenceexception-on-python-selenium). This can be resolved by refreshing the page and re-finding the element. I've added WebDriverWait .until to the main scraping element, "tbody", but the error still pops up occasionally. I'll add a try/except block to catch the error and refresh the page if it occurs.

Most elegant solution: It may be better to extract the data directly all the way down the tree to the "td" elements, rather than extracting the data at the "tbody" level and then the "tr" level and then iterating through the "td" elements. This would reduce the number of times we need to interact with the page and may reduce the likelihood of the error.

In [148]:
# Use Selenium to set up a chrome browser simulation with headless option so that we don't have fifty browser windows open
chrome_options = Options()
chrome_options.add_argument("--headless")  # Use --headless for older versions
browser = webdriver.Chrome(options=chrome_options) # Set up the browser

# Create an empty dictionary to store the paths to the CSV files for each event
csv_paths = {}

# Loop through the eventID_df and scrape the qualification data for each event given each event's URL
for index, row in eventID_df.iterrows():
    
    # Get the dataframe name for the current event from the "df_name" column
    df_name = eventID_df.at[index, "df_name"]

    # Stop loop if index is greater than 2 (for testing purposes) - remove the two lines below line for the final version
    # if index > 2:
        # break

    # browser = webdriver.Chrome(options=chrome_options)

    # Create an empty dataframe to store the qualification data for the event
    qual_df = pd.DataFrame(columns=["QP", "Nat", "Athlete", "Status", "Details"])

    # open the URL for the event in the current row
    url = row["Qual_URLs"]
    browser.get(url)
    
    # Wait for the page and all of its elements to load
    browser.implicitly_wait(20) # Seemed to throw an error for a 10 second waiting period so have increased it to 30s
    
    # Isolate the qualification data from the table with the class name "table"
    # qual_table = browser.find_element(By.TAG_NAME, "tbody")
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
    qual_table = WebDriverWait(browser, 10,ignored_exceptions=ignored_exceptions)\
                        .until(expected_conditions.presence_of_element_located((By.TAG_NAME, "tbody")))

    # Loop through the rows in the table and store the qualification data in the dataframe
    for row in qual_table.find_elements(By.TAG_NAME, "tr")[1:]:
        data = row.find_elements(By.TAG_NAME, "td")
        QP = data[0].text
        Nat = data[2].text
        Athlete = data[3].text
        Status = data[4].text
        Details = data[5].text
        qual_df = pd.concat([qual_df, pd.DataFrame([{"QP" : QP, "Nat" : Nat, "Athlete" : Athlete, "Status" : Status, "Details" : Details}])], ignore_index=True)

    # Save the qual_df dataframe to a CSV file as with the contents of df_name and overwrite any existing file
    qual_df.to_csv(f"CSV_dataframes/{df_name}.csv", index=False)    

    # Store the path to the CSV file in the csv_paths dictionary
    csv_paths[df_name] = f"CSV_dataframes/{df_name}.csv"

    # Print the first 5 rows of the dataframe to check the data
    print(qual_df.head())
    
# Close the browser
browser.quit()


  QP  Nat                   Athlete                       Status  \
0  1  JAM          Shericka JACKSON  Qualified by Entry Standard   
1  2  USA      Sha'Carri RICHARDSON  Qualified by Entry Standard   
2  3  CIV  Marie-Josée TA LOU-SMITH  Qualified by Entry Standard   
3  4  JAM   Shelly-Ann FRASER-PRYCE  Qualified by Entry Standard   
4  5  USA             Jacious SEARS  Qualified by Entry Standard   

                                                                    Details  
0           10.65 - (+1.0) - National Stadium, Kingston (JAM) - 07 JUL 2023  
1  10.65 - (-0.2) - Nemzeti Atlétikai Központ, Budapest (HUN) - 21 AUG 2023  
2              10.75 - (+1.2) - Olympic Stadium, London (GBR) - 23 JUL 2023  
3  10.77 - (-0.2) - Nemzeti Atlétikai Központ, Budapest (HUN) - 21 AUG 2023  
4   10.77 - (+1.6) - Percy Beard Track, Gainesville, FL (USA) - 13 APR 2024  
  QP  Nat               Athlete                       Status  \
0  1  JAM      Shericka JACKSON  Qualified by Entry Standar

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome-headless-shell=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x0115C1C3+27395]
	(No symbol) [0x010F3DC4]
	(No symbol) [0x00FF1B7F]
	(No symbol) [0x00FF6C7F]
	(No symbol) [0x00FF88F8]
	(No symbol) [0x00FF8970]
	(No symbol) [0x010328A7]
	(No symbol) [0x01032D3B]
	(No symbol) [0x01029011]
	(No symbol) [0x010539E4]
	(No symbol) [0x01028C15]
	(No symbol) [0x01053C34]
	(No symbol) [0x0106CB24]
	(No symbol) [0x01053736]
	(No symbol) [0x01027541]
	(No symbol) [0x010280BD]
	GetHandleVerifier [0x01413A93+2876371]
	GetHandleVerifier [0x01467F5D+3221661]
	GetHandleVerifier [0x011DD634+556916]
	GetHandleVerifier [0x011E474C+585868]
	(No symbol) [0x010FCE04]
	(No symbol) [0x010F9818]
	(No symbol) [0x010F99B7]
	(No

## Next steps

- For each event, extract the performance data for each event by qualifying athlete for 2023 and 2024. This will allow us to analyse and compare the athletes' performances in the two years leading up to the Olympics.
- Correct for wind conditions in the performance data for events where wind conditions can affect performance (e.g. track events).
- Model athlete performance data to predict the top athletes for each event in 2024.
- Modelling will involve determining an athlete's trajectory (e.g. improving, declining) relative to their personal best performance. The trajectory will be converted to a non-linear function (Gompertz growth model) to limit the athlete's performance as they approach/surpass their peak performance.
- Modelling will also involve determining an athlete's consistency from event to event. The consistency will be used to determine the uncertainty in the athlete's performance in 2024 and incorporated as a parameter in the Monte Carlo simulation (see below).
- The trajectory and consistency will be used to predict the athlete's performance in 2024 using a Monte Carlo simulation to account for uncertainty in the predictions.